In [11]:
from pycocotools.coco import COCO

# Path to your COCO annotation file
annotation_file = r"C:\Users\moham\Desktop\brain mri tumor detection\Axial-Dataset.v4-axial.coco\annotations\train.json"

# Load the annotation file
coco = COCO(annotation_file)

# Get all the categories (classes)
categories = coco.loadCats(coco.getCatIds())
class_names = [cat['name'] for cat in categories]
print("Classes:", class_names)

# Get information about the images and annotations
image_ids = coco.getImgIds()
images = coco.loadImgs(image_ids)
print(f"Number of images: {len(images)}")

# Get annotations for a specific image
img_id = image_ids[0]  # Select the first image as an example
annotation_ids = coco.getAnnIds(imgIds=[img_id])
annotations = coco.loadAnns(annotation_ids)

print("Annotations for the first image:", annotations)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Classes: ['tumor', 'negative', 'positive']
Number of images: 518
Annotations for the first image: [{'id': 0, 'image_id': 0, 'category_id': 2, 'bbox': [156, 222, 55, 70], 'area': 3850, 'segmentation': [], 'iscrowd': 0}]


In [8]:
import json

# Load the annotation file
with open(r"C:\Users\moham\Desktop\brain mri tumor detection\Axial-Dataset.v4-axial.coco\annotations\train.json", "r") as f:
    data = json.load(f)

# Update category IDs for 'tumor' and 'negative' to match 'positive'
# First, find the category ID for 'positive'
positive_category_id = None
for category in data['categories']:
    if category['name'] == 'positive':
        positive_category_id = category['id']
        break

if positive_category_id is None:
    raise ValueError("Category 'positive' not found in categories.")

# Update the annotations to make all 'tumor' and 'negative' annotations have the 'positive' ID
for ann in data['annotations']:
    if ann['category_id'] in [0, 1]:  # Assuming 0 is 'tumor' and 2 is 'negative'
        ann['category_id'] = positive_category_id

# Remove the 'tumor' and 'negative' categories from the categories list
data['categories'] = [cat for cat in data['categories'] if cat['name'] == 'positive']

# Save the updated annotations
with open(r"C:\Users\moham\Desktop\brain mri tumor detection\Axial-Dataset.v4-axial.coco\annotations\trainup.json", "w") as f:
    json.dump(data, f)

print("Classes merged successfully. Updated dataset saved.")

Classes merged successfully. Updated dataset saved.


In [16]:
from pycocotools.coco import COCO

# Path to your COCO annotation file
annotation_file = r"C:\Users\moham\Desktop\brain mri tumor detection\Axial-Dataset.v4-axial.coco\annotations\trainup.json"

# Load the annotation file
coco = COCO(annotation_file)

# Get all the categories (classes)
categories = coco.loadCats(coco.getCatIds())
class_names = [cat['name'] for cat in categories]
print("Classes:", class_names)

# Get information about the images and annotations
image_ids = coco.getImgIds()
images = coco.loadImgs(image_ids)
print(f"Number of images: {len(images)}")

# Get annotations for a specific image
img_id = image_ids[0]  # Select the first image as an example
annotation_ids = coco.getAnnIds(imgIds=[img_id])
annotations = coco.loadAnns(annotation_ids)

print("Annotations for the first image:", annotations)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Classes: ['positive']
Number of images: 518
Annotations for the first image: [{'id': 0, 'image_id': 0, 'category_id': 2, 'bbox': [156, 222, 55, 70], 'area': 3850, 'segmentation': [], 'iscrowd': 0}]


In [ ]:
# Load the annotation file
with open(r"C:\Users\moham\Desktop\brain mri tumor detection\Axial-Dataset.v4-axial.coco\annotations\test.json", "r") as f:
    data = json.load(f)

# Update category IDs for 'tumor' and 'negative' to match 'positive'
# First, find the category ID for 'positive'
positive_category_id = None
for category in data['categories']:
    if category['name'] == 'positive':
        positive_category_id = category['id']
        break

if positive_category_id is None:
    raise ValueError("Category 'positive' not found in categories.")

# Update the annotations to make all 'tumor' and 'negative' annotations have the 'positive' ID
for ann in data['annotations']:
    if ann['category_id'] in [0, 1]:  # Assuming 0 is 'tumor' and 2 is 'negative'
        ann['category_id'] = positive_category_id

# Remove the 'tumor' and 'negative' categories from the categories list
data['categories'] = [cat for cat in data['categories'] if cat['name'] == 'positive']

# Save the updated annotations
with open(r"C:\Users\moham\Desktop\brain mri tumor detection\Axial-Dataset.v4-axial.coco\annotations\testup.json", "w") as f:
    json.dump(data, f)

print("Classes merged successfully. Updated dataset saved.")

Classes merged successfully. Updated dataset saved.


In [2]:
import os
from detectron2.data.datasets import register_coco_instances
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
import detectron2

In [3]:
from detectron2.data import DatasetCatalog, MetadataCatalog

# Unregister existing datasets if they are already registered
for d in ["mri_train", "mri_test"]:
    if d in DatasetCatalog.list():
        DatasetCatalog.remove(d)
    if d in MetadataCatalog.list():
        MetadataCatalog.remove(d)

# Now re-register
register_coco_instances("mri_train", {}, r"C:\Users\moham\Desktop\brain mri tumor detection\Axial-Dataset.v4-axial.coco\annotations\trainup.json",  r"C:\Users\moham\Desktop\brain mri tumor detection\Axial-Dataset.v4-axial.coco\train")
register_coco_instances("mri_test", {}, r"C:\Users\moham\Desktop\brain mri tumor detection\Axial-Dataset.v4-axial.coco\annotations\testup.json",  r"C:\Users\moham\Desktop\brain mri tumor detection\Axial-Dataset.v4-axial.coco\test")


In [4]:
# Configuring the Fast R-CNN model
cfg = get_cfg()
cfg.merge_from_file(detectron2.model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("mri_train",)
cfg.DATASETS.TEST = ("mri_test",)  # For evaluation
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = detectron2.model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.001  # Learning rate
cfg.SOLVER.MAX_ITER = 300  # Adjust based on your needs
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1 # Tumor, positive

# Setup output directory
cfg.OUTPUT_DIR = "./output_mri"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

# Trainer setup
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

evaluator = COCOEvaluator("mri_test", cfg, False, output_dir=cfg.OUTPUT_DIR)
test_loader = build_detection_test_loader(cfg, "mri_test")  # No need to call it

[11/18 03:59:03 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, we

[11/18 03:59:03 d2.engine.train_loop]: Starting training from iteration 0


c:\Users\moham\anaconda3\envs\detectron_env\lib\site-packages\torch\functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3610.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[11/18 04:01:39 d2.utils.events]:  eta: 0:32:39  iter: 19  total_loss: 1.042  loss_cls: 0.6349  loss_box_reg: 0.3719  loss_rpn_cls: 0.0576  loss_rpn_loc: 0.006174    time: 7.2389  last_time: 6.2027  data_time: 0.3924  last_data_time: 0.0034   lr: 6.427e-05  max_mem: 2225M
[11/18 04:05:08 d2.utils.events]:  eta: 0:33:34  iter: 39  total_loss: 0.889  loss_cls: 0.3487  loss_box_reg: 0.4975  loss_rpn_cls: 0.02006  loss_rpn_loc: 0.005013    time: 8.9291  last_time: 5.5179  data_time: 0.0018  last_data_time: 0.0017   lr: 0.00013087  max_mem: 2225M
[11/18 04:07:38 d2.utils.events]:  eta: 0:31:14  iter: 59  total_loss: 0.9303  loss_cls: 0.314  loss_box_reg: 0.5891  loss_rpn_cls: 0.008421  loss_rpn_loc: 0.003967    time: 8.4427  last_time: 3.6884  data_time: 0.0018  last_data_time: 0.0024   lr: 0.00019747  max_mem: 2225M
[11/18 04:12:24 d2.utils.events]:  eta: 0:29:39  iter: 79  total_loss: 0.8802  loss_cls: 0.2585  loss_box_reg: 0.6199  loss_rpn_cls: 0.006943  loss_rpn_loc: 0.004422    time: 9

In [5]:
import random
import cv2
from detectron2.utils.visualizer import Visualizer
from detectron2.utils.visualizer import ColorMode

# Load the trained model
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Set a threshold for making predictions
predictor = detectron2.engine.DefaultPredictor(cfg)

# Load the test dataset
dataset_dicts = DatasetCatalog.get("mri_test")

# Visualize predictions on a few random test images
for d in random.sample(dataset_dicts, 3):  # Display predictions on 3 random images
    img = cv2.imread(d["file_name"])
    outputs = predictor(img)  # Run inference

    # Visualize the predictions
    v = Visualizer(img[:, :, ::-1],
                   metadata=MetadataCatalog.get("mri_test"), 
                   scale=0.8, 
                   instance_mode=ColorMode.IMAGE_BW  # Remove the color of the unsegmented parts
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    
    # Display the image with predictions
    cv2.imshow("Predicted Image", v.get_image()[:, :, ::-1])
    cv2.waitKey(0)


[11/18 04:51:09 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./output_mri\model_final.pth ...


c:\Users\moham\anaconda3\envs\detectron_env\lib\site-packages\fvcore\common\checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_locatio

WARNING [11/18 04:51:09 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[11/18 04:51:09 d2.data.datasets.coco]: Loaded 36 images in COCO format from C:\Users\moham\Desktop\brain mri tumor detection\Axial-Dataset.v4-axial.coco\annotations\testup.json
